In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = saw
band = [10]
commitnumber = de5d43e
molecule = {'h2o': 'atmpro'}
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band8
  atmpro = mls
  band = 8
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 0
  molecule = h2o
  ng_adju = [0, -1, 0]
  ng_refs = [3, 2, 3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250), (500, 250)]
  tsfc = 294
  vmax = 1900
  vmin = 1380
  w_diffuse = [(1.66, 1.66, 1.66), (1.66, 1.66), (1.66, 1.66, 1.66)]
  wgt = [(0.55, 0.55, 0.85), (0.85, 0.85), (0, 0.3, 0.55)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0 -0.201621                  NaN       NaN
0.000312      NaN       NaN                  1.0 -0.160407
0.000750      2.0 -0.193749                  2.0 -0.154516
0.001052      3.0 -0.173303                  3.0 -0.140934
0.001476      4.0 -0.150291                  4.0 -0.125356
0.002070      5.0 -0.124865                  5.0 -0.107759
0.002904      6.0 -0.096935                  6.0 -0.087915
0.004074      7.0 -0.066493                  7.0 -0.065567
0.005714      8.0 -0.033553                  8.0 -0.040640
0.008015      9.0  0.001777                  9.0 -0.013057
0.011243     10.0  0.039302                 10.0  0.017376
0.015771     11.0  0.078720                 11.0  0.050700
0.022122     12.0  0.119697                 12.0  0.086918
0.031031     13.0  0.161652                 13.0  0.125712
0.043528     14.0  0.195934                 14.0  0.160908
0.061057     15.0  0.213425                 15.0  0.185599
0.085645     16.0  0.220409                 16.0  0.203055
0.120136     17.0  0.223983                 17.0  0.217242
0.168516     18.0  0.225183                 18.0  0.227556
0.236378     19.0  0.225961                 19.0  0.234299
0.331549     20.0  0.230585                 20.0  0.240117
0.465100     21.0  0.245497                 21.0  0.248237
0.652400     22.0  0.206924                 22.0  0.204512
0.915100     23.0  0.117138                 23.0  0.112405
1.283650     24.0  0.057291                 24.0  0.052234
1.800600     25.0  0.029391                 25.0  0.029084
2.525700     26.0  0.016002                 26.0  0.019186
3.542800     27.0  0.008655                 27.0  0.011636
4.969550     28.0  0.006010                 28.0  0.007162
6.970850     29.0  0.006509                 29.0  0.006485
9.778100     30.0  0.006719                 30.0  0.006764
13.715850    31.0  0.005864                 31.0  0.006589
19.239350    32.0  0.004319                 32.0  0.005338
26.987250    33.0  0.003964                 33.0  0.004958
37.855300    34.0  0.005127                 34.0  0.006310
53.100050    35.0  0.006135                 35.0  0.007303
73.887500    36.0  0.007056                 36.0  0.007548
97.662500    37.0  0.007832                 37.0  0.007590
121.437500   38.0  0.007723                 38.0  0.007352
145.212500   39.0  0.009336                 39.0  0.009498
168.987500   40.0  0.010314                 40.0  0.012046
192.762500   41.0  0.010586                 41.0  0.014183
216.537500   42.0  0.009396                 42.0  0.013878
240.312500   43.0  0.007327                 43.0  0.011312
264.087500   44.0  0.003851                 44.0  0.006106
287.862500   45.0 -0.002597                 45.0 -0.003026
311.637500   46.0 -0.006069                 46.0 -0.008994
335.412500   47.0  0.000037                 47.0 -0.003927
359.187500   48.0  0.010978                 48.0  0.006916
382.962500   49.0  0.026120                 49.0  0.019514
406.737500   50.0  0.044861                 50.0  0.032925
430.512500   51.0  0.062143                 51.0  0.049902
454.287500   52.0  0.076478                 52.0  0.070162
478.062500   53.0  0.086983                 53.0  0.086719
501.837500   54.0  0.093528                 54.0  0.094538
525.612500   55.0  0.096452                 55.0  0.095831
549.387500   56.0  0.096344                 56.0  0.095005
573.162500   57.0  0.094025                 57.0  0.095637
596.937500   58.0  0.090111                 58.0  0.097024
620.712500   59.0  0.085063                 59.0  0.096156
644.487500   60.0  0.079200                 60.0  0.092540
668.262500   61.0  0.072800                 61.0  0.084611
692.037500   62.0  0.066184                 62.0  0.075241
715.812500   63.0  0.059721                 63.0  0.064754
739.587500   64.0  0.053961                 64.0  0.055321
76

# Fluxes by Layer

CRD                          CLIRAD                \
                         flug       fldg     fnetg       flug          fldg   
pressure     level                                                            
0.000000e+00 1      -4.883807   0.000000 -4.883807        NaN           NaN   
1.000000e-08 1            NaN        NaN       NaN  -4.889013  1.999387e-09   
6.244000e-04 2      -4.883828   0.000006 -4.883822  -4.889029  3.670858e-06   
8.759000e-04 3      -4.883836   0.000008 -4.883828  -4.889035  5.246083e-06   
1.228600e-03 4      -4.883847   0.000012 -4.883835  -4.889043  7.764129e-06   
1.723400e-03 5      -4.883862   0.000018 -4.883844  -4.889054  1.179112e-05   
2.417400e-03 6      -4.883881   0.000027 -4.883854  -4.889070  1.821787e-05   
3.390900e-03 7      -4.883907   0.000042 -4.883865  -4.889090  2.845760e-05   
4.756500e-03 8      -4.883942   0.000065 -4.883876  -4.889117  4.471726e-05   
6.672000e-03 9      -4.883986   0.000102 -4.883884  -4.889152  7.045367e-05   
9.358900e-03 10     -4.884042   0.000159 -4.883883  -4.889197  1.110813e-04   
1.312780e-02 11     -4.884113   0.000247 -4.883866  -4.889253  1.750298e-04   
1.841450e-02 12     -4.884198   0.000382 -4.883816  -4.889321  2.753009e-04   
2.583020e-02 13     -4.884297   0.000586 -4.883711  -4.889401  4.317556e-04   
3.623230e-02 14     -4.884405   0.000893 -4.883512  -4.889489  6.742402e-04   
5.082340e-02 15     -4.884515   0.001341 -4.883173  -4.889579  1.041858e-03   
7.129060e-02 16     -4.884624   0.001968 -4.882656  -4.889667  1.579913e-03   
1.000000e-01 17     -4.884724   0.002818 -4.881906  -4.889743  2.347437e-03   
1.402710e-01 18     -4.884790   0.003952 -4.880838  -4.889782  3.422216e-03   
1.967600e-01 19     -4.884769   0.005438 -4.879331  -4.889728  4.891566e-03   
2.759970e-01 20     -4.884561   0.007351 -4.877210  -4.889478  6.840440e-03   
3.871000e-01 21     -4.883947   0.009772 -4.874175  -4.888810  9.333563e-03   
5.431000e-01 22     -4.882393   0.012754 -4.869639  -4.887263  1.237347e-02   
7.617000e-01 23     -4.879824   0.015543 -4.864281  -4.884787  1.519451e-02   
1.068500e+00 24     -4.877047   0.017023 -4.860024  -4.882101  1.659400e-02   
1.498800e+00 25     -4.874435   0.017331 -4.857103  -4.879553  1.670836e-02   
2.102400e+00 26     -4.872012   0.017010 -4.855002  -4.877173  1.640799e-02   
2.949000e+00 27     -4.869818   0.016421 -4.853397  -4.875008  1.616752e-02   
4.136600e+00 28     -4.868014   0.015834 -4.852179  -4.873213  1.600923e-02   
5.802500e+00 29     -4.866648   0.015655 -4.850993  -4.871829  1.603903e-02   
8.139200e+00 30     -4.865504   0.016312 -4.849192  -4.870635  1.664075e-02   
1.141700e+01 31     -4.864508   0.017925 -4.846583  -4.869565  1.819638e-02   
1.601470e+01 32     -4.863932   0.020543 -4.843389  -4.868906  2.112670e-02   
2.246400e+01 33     -4.864556   0.024467 -4.840089  -4.869485  2.578363e-02   
3.151050e+01 34     -4.866880   0.031038 -4.835842  -4.871883  3.349640e-02   
4.420010e+01 35     -4.870524   0.042389 -4.828135  -4.875705  4.680397e-02   
6.200000e+01 36     -4.875458   0.060258 -4.815200  -4.880905  6.740304e-02   
8.577500e+01 37     -4.881166   0.085838 -4.795328  -4.886844  9.460069e-02   
1.095500e+02 38     -4.885607   0.112334 -4.773272  -4.891294  1.204280e-01   
1.333250e+02 39     -4.889689   0.138166 -4.751522  -4.895234  1.450755e-01   
1.571000e+02 40     -4.896302   0.171071 -4.725231  -4.901613  1.782053e-01   
1.808750e+02 41     -4.906455   0.210273 -4.696182  -4.911381  2.219025e-01   
2.046500e+02 42     -4.921596   0.255228 -4.666368  -4.925927  2.763956e-01   
2.284250e+02 43     -4.942360   0.302454 -4.639906  -4.946005  3.355617e-01   
2.522000e+02 44     -4.970556   0.351286 -4.619270  -4.973613  3.950296e-01   
2.759750e+02 45     -5.010657   0.402232 -4.608425  -5.013792  4.524060e-01   
2.997500e+02 46     -5.072874   0.457135 -4.615739  -5.077525  5.076150e-01   
3.235250e+02 47     -5.161689   0.528857 -4.632831  -5.169154  5.739106e-01 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')